In [1]:
from common import load_data

data, labels = load_data('src', 100)
print('Size of Data:', len(data))
print('Size of Label:', len(labels))

Size of Data: 100
Size of Label: 100


In [2]:
import preprocessor as pp
import numpy as np

dataset = [] # dataset
label = [] # label
for (index, image) in enumerate(data):
    normalized = pp.hist_normalize(image)
    houghed = pp.hough_circling(normalized)
    if houghed is not None:
        hog_img = pp.hog_descriptor(houghed)
        dataset.append(hog_img)
        label.append(labels[index])

print('dataset size:', len(dataset))
print('label size:', len(label))

dataset size: 88
label size: 88


In [3]:
from tensorflow.keras.utils import to_categorical

X = np.array(dataset)
X = X.reshape(*X.shape, 1)
label_set = set(label)
classes = { val: key + 1 for (key, val) in enumerate(label_set)}
Y = np.fromiter([classes[y] for y in label], dtype=np.int)
Y = to_categorical(Y)
dense = len(Y[0])
dense

4

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
X_train.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten

# initiate model
model = Sequential()

# add model layers
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(512, 256, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (4, 4), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (4, 4), activation='relu'))
model.add(Flatten())
model.add(Dense(dense, activation='softmax'))
model.summary()

In [ ]:
# compile model using accuracy to measure the performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

In [ ]:
model.save_weights('saved_models/my_model')